In [1]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import polars as pl
import nfl_data_py as nfl

env = "local"

In [2]:
if env == "local":
    os.chdir("/Users/samuel/Documents/GitHub/QB-GPT/")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")

In [3]:
os.listdir()

['data_models',
 '.DS_Store',
 'app',
 'LICENSE',
 'models',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'data_preprocessing',
 'index',
 '.git',
 'notebooks']

In [61]:
training_data = tf.data.Dataset.load("data_models/Helenos_categ/train_play_prediction_pois")
testing_data = tf.data.Dataset.load("data_models/Helenos_categ/test_play_prediction_pois")

In [5]:
train_length = [i for i,_ in enumerate(training_data)][-1] + 1
test_length = [i for i,_ in enumerate(testing_data)][-1] + 1

In [62]:
print("Train length is : ", str(train_length))
print("Test length is : ", str(test_length))

Train length is :  293150
Test length is :  125636


In [63]:
batch_size = 32

training_data = training_data.shuffle(train_length).batch(batch_size)
testing_data = testing_data.shuffle(test_length).batch(batch_size)

In [64]:
from models.modeling.QBGPT.models import QBGPT, AttentionBlock, AttentionBlockHelenos

moves_to_pred = 10876
input_size = 10878
starts_size = 1033
scrimmage_size = 100
positions_id = 29
temp_ids = 52

off_def_size = 2
token_type_size = 3
play_type_size = 9



model_tiny = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = off_def_size,
                    type_vocab_size = token_type_size,
                    playtype_vocab_size = play_type_size,
                    embedding_dim = 64,
                    hidden_dim = 64,
                    to_pred_size = moves_to_pred)



model_tiny.load_weights("models/modeling/QBGPT/weights/model_tiny/QBGPT")

In [65]:
qbgpt_data = tf.data.Dataset.load("data_models/QBGPT/test_tokens_NFL_GPT")

to_build = list(qbgpt_data.batch(32))[0][0]
to_build["token_type_ids"]

<tf.Tensor: shape=(32, 256), dtype=int64, numpy=
array([[0, 1, 1, ..., 2, 2, 2],
       [0, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 2, 2, 2],
       [0, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2]])>

In [66]:
model_tiny(to_build)

<tf.Tensor: shape=(32, 256, 10876), dtype=float32, numpy=
array([[[-31.184824, -31.321651, -33.827995, ..., -32.139767,
         -35.21571 , -32.04264 ],
        [-26.570581, -26.655142, -28.346746, ..., -27.595945,
         -30.311228, -27.650642],
        [-25.891638, -26.007206, -28.418224, ..., -26.820076,
         -29.752003, -26.752388],
        ...,
        [-18.545486, -18.92145 , -17.710766, ..., -18.594082,
         -19.279867, -18.179737],
        [-18.545486, -18.92145 , -17.710766, ..., -18.594082,
         -19.279867, -18.179737],
        [-18.545486, -18.92145 , -17.710766, ..., -18.594082,
         -19.279867, -18.179737]],

       [[-32.39939 , -32.851208, -35.903957, ..., -34.911182,
         -36.811886, -33.84422 ],
        [-32.60915 , -33.241703, -36.130302, ..., -35.601456,
         -36.518887, -34.383034],
        [-30.747688, -31.308493, -34.758984, ..., -33.396282,
         -34.767628, -32.17215 ],
        ...,
        [-20.650797, -21.537125, -21.102806, ..., 

In [67]:
# Define the original embedding layer with an index size of 28 and embedding dimension.
original_index_size = 29
embedding_dim = 64  # Change this to your desired embedding dimension.

# Create a new embedding layer with an index size of 29.
new_index_size = 30
new_embedding_layer = tf.keras.layers.Embedding(input_dim=new_index_size, output_dim=embedding_dim)

# Assume you have the original embedding weights as a NumPy array.
# Replace this with your actual weights.
original_embedding_weights = np.random.rand(original_index_size, embedding_dim)

# Copy weights from the original embedding layer to the new one, preserving 28 weights.
new_embedding_weights = np.zeros((new_index_size, embedding_dim))
new_embedding_weights[:original_index_size, :] = original_embedding_weights

# Set the weights of the new index (29) to the 21st weight.
new_embedding_weights[new_index_size - 1, :] = original_embedding_weights[4, :]

# Set the weights of the new embedding layer to the modified weights.
new_embedding_layer.build((None,))
new_embedding_layer.set_weights([new_embedding_weights])

In [68]:
model_tiny.Encoder.Embedding.PositionEmbedding.Embedding = new_embedding_layer

In [69]:
tfd = tfp.distributions
tfpl = tfp.layers

In [70]:
from typing import List, Optional, Union
import numpy as np

def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    """
    Deal with dynamic shape in tensorflow cleanly.

    Args:
        tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.

    Returns:
        `List[int]`: The shape of the tensor as a list.
    """
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [71]:
class AttentionBlockHelenos(tf.keras.Model):
  def __init__(self,
               num_heads : int,
               hidden_dim : int,
               output_dim : int):
        super(AttentionBlockHelenos, self).__init__()

        self.num_attention_heads = num_heads
        self.attention_head_size = hidden_dim
        self.total_dim = num_heads * hidden_dim
        self.output_dim = output_dim
        
        self.NormIn = tf.keras.layers.LayerNormalization(name = "Norm_in")
        self.Query = tf.keras.layers.Dense(self.total_dim, name = "Query")
        self.Key = tf.keras.layers.Dense(self.total_dim, name = "Key")
        self.Value = tf.keras.layers.Dense(self.total_dim, name = "Value")
        self.DenseAtt = tf.keras.layers.Dense(output_dim, name = "Dense", activation = "relu")
        
        self.Add = tf.keras.layers.Add(name = "Add")
        self.Drop = tf.keras.layers.Dropout(rate = 0.1)
        self.DenseOut = tf.keras.layers.Dense(output_dim, name = "Dense", activation = "relu")
        self.NormOut = tf.keras.layers.LayerNormalization(name = "Norm_out")

  def transpose_for_scores(self, tensor: tf.Tensor, batch_size: int) -> tf.Tensor:
        # Reshape from [batch_size, seq_length, all_head_size] to [batch_size, seq_length, num_attention_heads, attention_head_size]
        tensor = tf.reshape(tensor=tensor, shape=(batch_size, -1, self.num_attention_heads, self.attention_head_size))

        # Transpose the tensor from [batch_size, seq_length, num_attention_heads, attention_head_size] to [batch_size, num_attention_heads, seq_length, attention_head_size]
        return tf.transpose(tensor, perm=[0, 2, 1, 3])

  def compute_scaled_attn_scores(self, query, key):
    attention_scores = tf.matmul(query, key, transpose_b=True)  # Transpose the second sequence

    # If you want scaled dot-product attention, divide by the square root of the embedding dimension
    embedding_dim = query.shape[-1]
    scaled_attention_scores = attention_scores / tf.math.sqrt(tf.cast(embedding_dim, dtype=tf.float32))

    return scaled_attention_scores

  def call(self,
           q : tf.Tensor,
           k : tf.Tensor,
           v : tf.Tensor):

    batch_size = shape_list(q)[0]
    
    norm_hidden_states_q = self.NormIn(q)
    norm_hidden_states_k = self.NormIn(k)
    norm_hidden_states_v = self.NormIn(v)
    
    query = self.Query(norm_hidden_states_q)
    queries = self.transpose_for_scores(query, batch_size)

    key = self.Key(norm_hidden_states_k)
    keys = self.transpose_for_scores(key, batch_size)
    
    value = self.Key(norm_hidden_states_v)
    values = self.transpose_for_scores(value, batch_size)
    
    attention_weights = self.compute_scaled_attn_scores(queries, keys)
    
    attention_scores = tf.matmul(attention_weights, values)
    attention_scores = tf.transpose(attention_scores, perm=[0, 2, 1, 3])
    attention_scores = tf.reshape(tensor=attention_scores, shape=(batch_size, -1, self.total_dim))
    attention_scores = self.DenseAtt(attention_scores)
    
    output = self.Add([attention_scores, norm_hidden_states_v])
    
    norm_output = self.NormOut(output)
    densed_output = self.DenseOut(norm_output)
    
    output = self.Add([densed_output, output])
    
    output = self.Drop(output)
    return output

In [72]:
class SimpleAttentionBlock(tf.keras.Model):
  def __init__(self,
               hidden_dim : int,
               output_dim : int):
        super(SimpleAttentionBlock, self).__init__()

        self.num_attention_heads = 1
        self.attention_head_size = hidden_dim
        self.total_dim = 1 * hidden_dim
        self.output_dim = output_dim
        
        self.NormIn = tf.keras.layers.LayerNormalization(name = "Norm_in")
        self.Query = tf.keras.layers.Dense(self.total_dim, name = "Query")
        self.Key = tf.keras.layers.Dense(self.total_dim, name = "Key")
        self.Value = tf.keras.layers.Dense(self.total_dim, name = "Value")
        self.DenseAtt = tf.keras.layers.Dense(output_dim, name = "Dense", activation = "relu")
        
        self.Add = tf.keras.layers.Add(name = "Add")
        self.Drop = tf.keras.layers.Dropout(rate = 0.1)
        self.DenseOut = tf.keras.layers.Dense(output_dim, name = "Dense", activation = "relu")
        self.NormOut = tf.keras.layers.LayerNormalization(name = "Norm_out")

  def compute_scaled_attn_scores(self, query, key):
    attention_scores = tf.matmul(query, key, transpose_b=True)  # Transpose the second sequence

    # If you want scaled dot-product attention, divide by the square root of the embedding dimension
    embedding_dim = query.shape[-1]
    scaled_attention_scores = attention_scores / tf.math.sqrt(tf.cast(embedding_dim, dtype=tf.float32))

    return scaled_attention_scores

  def call(self,
           q : tf.Tensor,
           k : tf.Tensor,
           v : tf.Tensor):

    batch_size = shape_list(q)[0]
    
    norm_hidden_states_q = self.NormIn(q)
    norm_hidden_states_k = self.NormIn(k)
    norm_hidden_states_v = self.NormIn(v)
    
    queries = self.Query(norm_hidden_states_q)

    keys = self.Key(norm_hidden_states_k)
    
    values = self.Key(norm_hidden_states_v)
    
    attention_weights = self.compute_scaled_attn_scores(queries, keys)
    
    attention_scores = tf.matmul(attention_weights, values)
    attention_scores = self.DenseAtt(attention_scores)
    
    output = self.Add([attention_scores, norm_hidden_states_v])
    
    norm_output = self.NormOut(output)
    densed_output = self.DenseOut(norm_output)
    
    output = self.Add([densed_output, output])
    
    output = self.Drop(output)
    return output

In [83]:
class DownEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(DownEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["down_ID"])
    return embed

class SeasonEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(SeasonEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["season_ID"])
    return embed

class TeamEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(TeamEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["team_ID"])
    return embed

class PlayerEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(PlayerEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["player_ids"])
    return embed


class MetaEmbedding(tf.keras.Model):
  def __init__(self, 
               team_vocab_size : int, 
               player_vocab_size : int, 
               season_vocab_size : int, 
               down_vocab_size : int, 
               embedding_dim : int):
        super(MetaEmbedding, self).__init__()
        
        self.embedding_dim = embedding_dim

        self.TeamEmbedding = TeamEncoder(vocab_size= team_vocab_size,
                                         embedding_dim=embedding_dim)
        self.PlayerEmbedding = PlayerEncoder(vocab_size= player_vocab_size,
                                             embedding_dim=embedding_dim)
        self.SeasonEmbedding = SeasonEncoder(vocab_size= season_vocab_size,
                                             embedding_dim=embedding_dim)
        self.DownEmbedding = DownEncoder(vocab_size= down_vocab_size,
                                         embedding_dim=embedding_dim)
        
        self.Attention = SimpleAttentionBlock(hidden_dim=embedding_dim,
                                              output_dim=embedding_dim)
        
        self.Conc = tf.keras.layers.Concatenate(axis = 2)
        
        self.Dense = tf.keras.layers.Dense(embedding_dim)

  def call(self, x):
    team_embed = self.TeamEmbedding(x)
    team_embed = tf.expand_dims(team_embed, axis = 2)
    
    player_embed = self.PlayerEmbedding(x)
    player_embed = tf.expand_dims(player_embed, axis = 2)
    
    season_embed = self.SeasonEmbedding(x)
    season_embed = tf.expand_dims(season_embed, axis = 2)
    
    down_embed = self.DownEmbedding(x)
    down_embed = tf.expand_dims(down_embed, axis = 2)
    
    added = self.Conc([team_embed, player_embed, season_embed, down_embed])
    
    logits = self.Attention(added, added, added)
    logits = tf.reshape(tensor=logits, shape=(batch_size, 11, -1, self.embedding_dim * 4))
    logits = tf.squeeze(logits, axis = 2)
    
    encoded = self.Dense(logits)
    
    return encoded

class Helenos(tf.keras.Model):
    def __init__(self, 
                 team_vocab_size : int, 
                 player_vocab_size : int,
                season_vocab_size : int,
                down_vocab_size : int, 
                embedding_dim : int,
                encoder : tf.keras.Model):
        super(Helenos, self).__init__()
        
        self.Encoder = encoder
        
        self.MetaEmbedding = MetaEmbedding(team_vocab_size=team_vocab_size,
                                           player_vocab_size = player_vocab_size,
                                           season_vocab_size=season_vocab_size,
                                           down_vocab_size=down_vocab_size,
                                           embedding_dim=embedding_dim)
        
        self.Add = tf.keras.layers.Add()
        
        self.Concat2 = tf.keras.layers.Concatenate(axis = -2)
        
        self.Attention = AttentionBlockHelenos(num_heads=3,
                                               hidden_dim=64,
                                               output_dim=embedding_dim)
        
        self.Concat3 = tf.keras.layers.Concatenate(axis = -1)
        
        self.Average = tf.keras.layers.GlobalAveragePooling1D()
        
        self.Params = tf.keras.layers.Dense(1)
        
        self.Dist = tfpl.DistributionLambda(lambda t:
                    tfd.Poisson(rate = tf.math.softplus(t[...,0])))
        
        
    
    def call(self, x):
        encoded_qb_off = self.Encoder(x["off"])
        encoded_meta_off = self.MetaEmbedding(x["off"])
        
        encoded_qb_def = self.Encoder(x["def"])
        encoded_meta_def = self.MetaEmbedding(x["def"])
        
        logits_off = self.Attention(q = encoded_qb_off, k = encoded_meta_off, v = encoded_meta_off)
        logits_def = self.Attention(q = encoded_qb_def, k = encoded_meta_def, v = encoded_meta_def)
        
        logits = self.Concat2([logits_off, logits_def])
        
        logits = self.Average(logits)
        params = self.Params(logits)
        
        dist = self.Dist(params)
        
        return dist

In [84]:
model = Helenos(team_vocab_size=32,
                player_vocab_size=7_226,
                season_vocab_size= 7,
                down_vocab_size= 5,
                embedding_dim= 64,
                encoder = model_tiny.Encoder)

model.Encoder.trainable = False

In [85]:
list(testing_data)[0][1]

<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([110., 111., 107., 105., 104., 102., 101., 105.,  98., 100., 120.,
        99.,  99., 122., 104.,  96., 109., 111.,  99.,  98., 155.,  97.,
        99.,  99., 101., 102., 105.,  99., 106., 103., 102., 109.])>

In [86]:
essai = list(testing_data)[0][0]
model(essai)

<tfp.distributions._TensorCoercible 'tensor_coercible' batch_shape=[32] event_shape=[] dtype=float32>

In [87]:
negloglik = lambda y, rv_y: -rv_y.log_prob(y)

In [88]:
def scheduler(epoch, lr):
  if epoch < 1:
      return 1e-4
  else:
      return 5e-5


schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [89]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss= negloglik)

history_small = model.fit(training_data, epochs=2)

Epoch 1/2


2023-10-09 23:21:19.118462: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:79: Filling up shuffle buffer (this may take a while): 182427 of 293150


  15/9161 [..............................] - ETA: 2:00 - loss: 87.3030   

2023-10-09 23:21:25.263220: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


9159/9161 [============================>.] - ETA: 0s - loss: 3.7808

InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/lambda/tensor_coercible_CONSTRUCTED_AT_helenos_6_distribution_lambda_6/log_prob/multiply_no_nan/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/samuel/anaconda3/envs/nflgpt/lib/python

In [71]:
X_Y_test = list(testing_data)[0]
X_test = X_Y_test[0]
Y_test = X_Y_test[1].numpy()
pred_dist = model(X_test)

In [72]:
loc_hat = np.mean(pred_dist.parameters["loc"])
scale_hat = np.mean(pred_dist.parameters["scale"])
rate_hat = np.mean(pred_dist.parameters["rate"])

In [73]:
pred_dist.parameters

{'loc': <tf.Tensor: shape=(32,), dtype=float32, numpy=
 array([-1.9706415 , -0.27730155, -1.8973212 , -0.12310744, -2.0525064 ,
        -1.9909205 , -2.0068512 , -0.04371835,  0.03401811, -1.853869  ,
        -1.9749445 , -0.12070109, -1.942014  , -1.8648635 , -0.34762996,
        -2.1113677 , -2.5591092 , -0.11513726, -1.9697163 , -2.9186897 ,
        -2.2004464 , -3.0807996 , -0.28684127, -0.02177463, -0.10883667,
        -1.9541225 , -2.0828385 , -0.11423492, -2.6962228 , -0.13184486,
        -2.6722188 , -0.00380048], dtype=float32)>,
 'scale': <tf.Tensor: shape=(32,), dtype=float32, numpy=
 array([3.2900844, 1.967712 , 3.1387942, 2.1550481, 3.250607 , 3.2937913,
        3.3023915, 2.1097374, 2.146712 , 3.146023 , 3.28955  , 2.1755946,
        3.2438848, 3.1593826, 1.9089994, 3.4005303, 3.6838508, 2.1110296,
        3.2780778, 3.9324925, 3.4529335, 4.07618  , 1.8737396, 2.0216503,
        2.2148094, 3.301769 , 3.3243136, 2.1905463, 3.7907004, 2.0425477,
        3.8286166, 2.1850023

In [66]:
df

,Class,Percentage
0,0,34.375
1,3,9.375
2,2,6.250
3,4,6.250
4,12,3.125
5,28,3.125
6,7,3.125
7,17,3.125
8,10,3.125
9,-3,3.125


In [67]:
import numpy as np
import plotly.graph_objs as go
from scipy.stats import exponnorm
from scipy.optimize import curve_fit

def exponnorm_pdf(x, loc, scale, K):
    return exponnorm.pdf(x, K, loc=loc, scale=scale)

value_counts = pd.Series(Y_test.astype("int32")).value_counts(normalize=True)

# Create a DataFrame for plotting
df = pd.DataFrame({'Class': value_counts.index, 'Percentage': value_counts.values})

# Create a figure
fig = go.Figure()
x = np.linspace(-20, 20, 1000)

# Add the line plot
fig.add_trace(go.Scatter(x=x, y=exponnorm_pdf(x, loc = loc_hat, scale = scale_hat, K = rate_hat), 
                         mode='lines', name='Fitted Exponnorm PDF', line=dict(color='red')))

# Add the bar plot
fig.add_trace(go.Bar(x=df['Class'], y=df['Percentage'], name='Bar Plot'))

# Update layout and labels
fig.update_layout(title='Combined Line and Bar Plot', xaxis=dict(title='X-Axis'), yaxis=dict(title='Y-Axis'))

# Show the combined plot
fig.show()

In [60]:
import numpy as np
import plotly.graph_objs as go
from scipy.stats import exponnorm
from scipy.optimize import curve_fit

# Define a function for the PDF of the fitted distribution
def exponnorm_pdf(x, loc, scale, K):
    return exponnorm.pdf(x, K, loc=loc, scale=scale)

# Create a range of x values for the plot
x = np.linspace(-20, 20, 1000)

# Create a scatter trace for the PDF curve of the fitted distribution
pdf_trace = go.Scatter(x=x, y=exponnorm_pdf(x, loc = loc_hat, scale = scale_hat, K = rate_hat), mode='lines', name='Fitted Exponnorm PDF', line=dict(color='red'))

# Create the layout for the plot
layout = go.Layout(title='Sampled Data and Fitted Exponnorm Distribution',
                   xaxis=dict(title='X-axis'),
                   yaxis=dict(title='Probability Density'))

# Create a figure and add the traces to it
fig = go.Figure(data=[pdf_trace], layout=layout)

# Display the plot in Jupyter Notebook or as an HTML file
fig.show()